In [ ]:
import requests
import json
import pandas as pd

In [ ]:
#Load in Protected Keys
with open(r'C:\Users\G-Unit\Chris\NSSFiles\Capstone\Data\API_Keys\OTClient_ID.json') as fi:
    credentials = json.load(fi)
with open(r'C:\Users\G-Unit\Chris\NSSFiles\Capstone\Data\API_Keys\OTSecret.json') as fi2:
    credentials2 = json.load(fi2)

In [ ]:
# Define Credentials, Base URL
base_url = 'https://platform.opentable.com/sync/v2'
client_id= credentials['client_id']
client_secret=credentials2['client_secret']
grant_type='client_credentials'
# Request Access Token
response = requests.get('https://oauth.opentable.com/api/v2/oauth/token?grant_type=client_credentials',
                        auth=(client_id, client_secret),
                         data={'grant_type':grant_type,'client_id':client_id,'client_secret':client_secret})
print(response.json())
response

In [ ]:
# Import Protected Bearer Token
with open(r'C:\Users\G-Unit\Chris\NSSFiles\Capstone\Data\API_Keys\OTAccess.json') as fi3:
    credentials3 = json.load(fi3)
# Use Bearer Token to Call API endpoint
access_token = credentials3['access_token']
endpoint=(base_url+'/reservations?rid=168415&scheduled_time_from=2023-04-01T00:00:00&scheduled_time_to=2023-04-30T23:59:59')
# Label Headers
headers ={'Authorization': f'Bearer{access_token}',
         'Content-Type': 'application/json'}
response2=requests.get(endpoint, headers=headers)
print(response2)

In [ ]:
# Call API while looking to see if 'nextPageUrl' is empty to get all results rather than the natural API limit of 2000.
response2_json=requests.get(endpoint, headers=headers).json()
reservations=response2_json['items']
while response2_json['nextPageUrl']:
# Using bearer token again to call 'nextPageUrl'    
    response2_json=requests.get(response2_json['nextPageUrl'],headers=headers).json()
    reservations.extend(response2_json['items'])
print(reservations)

In [ ]:
# Write to DF then to CSV
reservations=[]
item_json=response2.json()["items"]
reservations.extend(item_json)
df=pd.DataFrame(reservations)
df.to_csv(r'C:\Users\G-Unit\Chris\NSSFiles\Capstone\SGT.csv')